<div class='bar_title'></div>

*Introduction to Data Science*

# Modeling: Part b)

Gunther Gust<br>
Chair of Enterprise AI

Winter Semester 25/26

<img src="https://raw.githubusercontent.com/GuntherGust/tds2_data/main/images/d3.png?raw=1" style="width:20%; float:left;" />

<img src="https://raw.githubusercontent.com/vhaus63/ids_data/main/ao_modeling.png" style="width:80%" />

# Content

* Predictive Models
    - Random Forests
    - Boosting Methods
* Unsupervised Learning
    - k-Means Clustering
    - Other clustering techniques

Credits: The implementation data examples are adopted from www.kaggle.com.

# Random Forests


Decision Trees (CART) are:
- Easy to interpret  
- Flexible  
- Able to model nonlinear relationships  

But they also have a major weakness:
- __They have extremely high variance:__
    - Small changes in the training data → very different trees  
    - Deep trees can perfectly fit the training data → **overfitting**  
    
High variance harms generalization performance.



In contrast, a shallow tree has **lower variance** but **high bias** (underfitting, the model is "biased" in certain regions of the data)  

It is hard to tune a single tree to achieve __both low bias *and* low variance__, but:

> How about combining many (smaller) trees and average their predictions to reduce variance?

This is exactly what __random forests:__ do:

<img src="https://raw.githubusercontent.com/GuntherGust/tds2_data/main/images/03/Random_forest_explain.png?raw=1" style="width:80%; float:left;" />


## Ensemble Learning

A Random Forest is an **ensemble** of decision trees.

Key idea:
- “Many weak learners can create a strong learner.”

Benefit:
- Individual errors tend to cancel out.
- Forest becomes more stable and robust.

### Example: Why Ensembles Improve Accuracy

**Task:** Email classification (SPAM / NOT SPAM)

- We have **3 uncorrelated models**, each with accuracy **70%**.
- The ensemble prediction is made by **majority vote** of the 3 models:


| Outputs of three models     | Probability                        | Ensemble’s output |
|-----------------------------|-------------------------------------|-------------------|
| All three are correct       | `0.7 * 0.7 * 0.7 = 0.343`           | Correct           |
| Only two are correct        | `(0.7 * 0.7 * 0.3) * 3 = 0.441`     | Correct           |
| Only one is correct         | `(0.3 * 0.3 * 0.7) * 3 = 0.189`     | Wrong             |
| None are correct            | `0.3 * 0.3 * 0.3 = 0.027`           | Wrong             |


- Probability that at least **2 models** are correct:

        0.343 + 0.441 = 0.784 
        
- Ensembling __improves accuracy by 8.4%__ 




__Important note:__ The example does not take into account the fact that in real-world applications individial learners are __never perfectly uncorrelated__ - which reduces the gains of ensembling. 

## Training: Bagging (Bootstrap Aggregation)

1. __Bootstrapping__ for building the trees:
    1. Draw a **bootstrap sample** of observations from the training data (= sample with replacement)
    2. Train a CART tree on that sample
    3. Repeat for many trees

> __Effect:__ Each tree sees __*slightly different data*__ and their predictions errors are __*less correlated*__

2. __Aggregation__ for predictions:
- **Classification:** majority vote of all trees  
- **Regression:** average of predictions of all trees

## Random Feature Selection (Feature Bagging)

In addition to bagging, Random Forests randomize the **feature selection** at each split.

At each node:
- Consider only a **random subset** of the overall p features  
  - Classification: often `sqrt(p)`  
  - Regression: often `p/3`

Benefits:
- Trees become more **decorrelated**  
- No single strong predictor dominates the forest  
- Improves generalization

## Hyperparameters

### Model Size
- `n_estimators`: number of trees  
- More trees → better performance (to a point)

### Randomization
- `max_features`: number of features considered at each split  
- `bootstrap`: whether bootstrapping for the trainings samples is used (usually True)

### Sampling
- `max_samples`: size of bootstrap sample  


A solution to this problem is a procedure called cross-validation (CV for short). A test set should still be held out for final evaluation, but the validation set is no longer needed when doing CV. In the basic approach, called k-fold CV, the training set is split into k smaller sets (other approaches are described below, but generally follow the same principles). The following procedure is followed for each of the k “folds”:

- A model is trained using k-1 of the folds as training data;
- the resulting model is validated on the remaining part of the data (i.e., it is used as a test set to compute a performance measure such as accuracy).

The performance measure reported by k-fold cross-validation is then the average of the values computed in the loop. This approach can be computationally expensive, but does not waste too much data (as is the case when fixing an arbitrary validation set), which is a major advantage in problems such as inverse inference where the number of samples is very small.

<img src="https://raw.githubusercontent.com/vhaus63/ids_data/main/kfoldcv.png" style="width:70%" />

## Feature Importance

Random Forests provide feature importance estimates:

### 1. Mean Decrease Impurity (MDI)
- Based on how much each feature reduces impurity across splits.

### 2. Permutation Importance
- Shuffle the values of each feature and measure performance drop.
- More reliable and intuitive.

> If shuffling a feature hurts the model, it must be important.

## Strengths of Random Forests

- __Very important:__ High accuracy with minimal tuning  
- Robust against overfitting  
- Handles nonlinearities and feature interactions  
- Works well with mixed data types  
- Provides feature importance  
- No need for feature scaling  


## Weaknesses of Random Forests

- Less interpretable than a single tree  
- Large models (memory-heavy)  
- Slower predictions than individual trees  
- Struggles with very high-dimensional sparse data (e.g., text)  

## Implementing a Random Forest
Thanks to our Scikit-learn modeling pipeline we can reuse most of our code to train a random forest model with 100 trees.

In [1]:
import pandas as pd
import sklearn

In [2]:
melbourne_file_path = 'https://raw.githubusercontent.com/vhaus63/ids_data/refs/heads/main/melb_data.csv'
melbourne_data = pd.read_csv(melbourne_file_path)

melbourne_data.head()

,Unnamed: 0,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
3,3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,...,2.0,1.0,94.0,NaN,NaN,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0
4,4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,...,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0


In [3]:
melbourne_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     13580 non-null  int64  
 1   Suburb         13580 non-null  object 
 2   Address        13580 non-null  object 
 3   Rooms          13580 non-null  int64  
 4   Type           13580 non-null  object 
 5   Price          13580 non-null  float64
 6   Method         13580 non-null  object 
 7   SellerG        13580 non-null  object 
 8   Date           13580 non-null  object 
 9   Distance       13580 non-null  float64
 10  Postcode       13580 non-null  float64
 11  Bedroom2       13580 non-null  float64
 12  Bathroom       13580 non-null  float64
 13  Car            13518 non-null  float64
 14  Landsize       13580 non-null  float64
 15  BuildingArea   7130 non-null   float64
 16  YearBuilt      8205 non-null   float64
 17  CouncilArea    12211 non-null  object 
 18  Lattit

For simplicity we remove rows with missing values for this example. Note that a missing value can sometimes be a valuable information.

In [4]:
print(melbourne_data.shape)
melbourne_data.dropna(axis=0, inplace=True)
print(melbourne_data.shape)

(13580, 22)
(6196, 22)


In [5]:
y = melbourne_data['Price']
y.describe()

count    6.196000e+03
mean     1.068828e+06
std      6.751564e+05
min      1.310000e+05
25%      6.200000e+05
50%      8.800000e+05
75%      1.325000e+06
max      9.000000e+06
Name: Price, dtype: float64

In [6]:
melbourne_features = ['Rooms', 'Bathroom', 'Landsize', 'BuildingArea',
                        'YearBuilt', 'Lattitude', 'Longtitude']

In [7]:
X = melbourne_data[melbourne_features]

In [8]:
print(X.shape)
X.head()

(6196, 7)


,Rooms,Bathroom,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude
1,2,1.0,156.0,79.0,1900.0,-37.8079,144.9934
2,3,2.0,134.0,150.0,1900.0,-37.8093,144.9944
4,4,1.0,120.0,142.0,2014.0,-37.8072,144.9941
6,3,2.0,245.0,210.0,1910.0,-37.8024,144.9993
7,2,1.0,256.0,107.0,1890.0,-37.8060,144.9954


In [9]:
X.describe().T

,count,mean,std,min,25%,50%,75%,max
Rooms,6196.0,2.931407,0.971079,1.00000,2.000000,3.00000,4.0000,8.00000
Bathroom,6196.0,1.576340,0.711362,1.00000,1.000000,1.00000,2.0000,8.00000
Landsize,6196.0,471.006940,897.449881,0.00000,152.000000,373.00000,628.0000,37000.00000
BuildingArea,6196.0,141.568645,90.834824,0.00000,91.000000,124.00000,170.0000,3112.00000
YearBuilt,6196.0,1964.081988,38.105673,1196.00000,1940.000000,1970.00000,2000.0000,2018.00000
Lattitude,6196.0,-37.807904,0.075850,-38.16492,-37.855438,-37.80225,-37.7582,-37.45709
Longtitude,6196.0,144.990201,0.099165,144.54237,144.926198,144.99580,145.0527,145.52635


In [10]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

import numpy as np
import matplotlib.pyplot as plt

In [11]:
# Split data
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, train_size=0.8, test_size=0.2, random_state=0
)

# -----------------------------
# Decision Tree Model
# -----------------------------
tree_model = DecisionTreeRegressor(random_state=1)
tree_model.fit(X_train, y_train)
tree_preds = tree_model.predict(X_valid)
tree_mae = mean_absolute_error(y_valid, tree_preds)

# -----------------------------
# Random Forest Model
# -----------------------------
forest_model = RandomForestRegressor(
    random_state=1,
    n_estimators=100
)
forest_model.fit(X_train, y_train)
forest_preds = forest_model.predict(X_valid)
forest_mae = mean_absolute_error(y_valid, forest_preds)

# -----------------------------
# Compare performance
# -----------------------------
print("Decision Tree MAE:     {:.3f}".format(tree_mae))
print("Random Forest MAE:     {:.3f}".format(forest_mae))


Decision Tree MAE:     253848.377
Random Forest MAE:     193739.859


There is likely room for further improvement, but this is a __big improvement over the decision tree error of 253,000__. There are parameters which allow you to change the performance of the Random Forest much as we changed the maximum depth of the single decision tree. But one of the best features of Random Forest models is that they generally work reasonably even __without this tuning.__

Finally, let's use the built-in __feature importance__ method to see which features drive real estate prices in our example:

In [12]:
importances = forest_model.feature_importances_

# Put into a DataFrame for nice display
feat_importances = pd.DataFrame(
    {"feature": feature_names, "importance": importances}
).sort_values("importance", ascending=False)

plt.figure(figsize=(8, 5))
plt.bar(feat_importances["feature"], feat_importances["importance"])
plt.xticks(rotation=45, ha="right")
plt.ylabel("Importance")
plt.title("Random Forest Feature Importances")
plt.tight_layout()
plt.show()

NameError: name 'feature_names' is not defined

## Excursus: Beyond Feature Importance — Explainable Machine Learning

Random Forests give us **feature importance values**, but these have limitations:
- They show **how important** a feature is,  
- but not **whether it increases or decreases the prediction**,  
- and not **how the effect changes across different samples**.

There are several __explainability methods__ that can be used with any model and that go further:

- **Permutation Importance**: Measures how performance drops when a feature is shuffled.
 
- **Partial Dependence Plots (PDP)**: Show how the *average* prediction changes with a feature.

- **Individual Conditional Expectation (ICE)**: Shows how predictions change for *individual samples*.

- **LIME (Local Interpretable Model-Agnostic Explanations)**: Explains individual predictions using simple local surrogate models.

- **SHAP(SHapley Additive exPlanations)** : 
    - Shows the **direction** of each feature’s effect (positive / negative),
    - Quantifies how much each feature contributes to **individual predictions**,
    - Separates **global** and **local** explanations,
    - Handles nonlinear interactions naturally.

## More on random forests


- For an informal and illustrative introduction, see [this youtube video](https://www.youtube.com/watch?v=J4Wdy0Wc_xQ&ab_channel=StatQuestwithJoshStarmer). 
- For a formal coverage see [Hastie et al: Elements of Statistical Learning, Chapter 15](https://www.sas.upenn.edu/~fdiebold/NoHesitations/BookAdvanced.pdf)


# Mentimeter

Let's recap what we have learned on random forests on [Mentimeter](https://www.menti.com/alurte2x2dno). 

# Boosting algorithms

## Random forest: Bagging 

<img src="https://raw.githubusercontent.com/vhaus63/ids_data/main/bagging.png" style="width:60%" />

## Another powerful approach: Boosting

<img src="https://raw.githubusercontent.com/vhaus63/ids_data/main/boosting.png" style="width:70%" />

## What Boosting Does (Intuition)

Boosting builds a model **sequentially**:

1. Train a weak model (usually a very small tree).
2. Identify what it got wrong.
3. Train the next tree to fix these errors.
4. Repeat many times.
5. Combine all trees into one strong predictor.

**Boosting = “Learning from your mistakes.”**

Unlike Random Forests (parallel), boosting trees grow **one after another**,  
each improving the previous model.

## Boosting for regression:

- The next learner fits the **residuals** previous learner (remaining errors).
- Each step reduces the leftover error.

## Boosing for classification:
- Misclassified points implicitly get more influence.
- Hard examples are emphasized.


## Example: Adaboost (Adaptive Boosting)

- Start with equal weights for all training samples.
- After each tree:
  - Misclassified samples get **more weight**.
  - Correctly classified samples get **less weight**.
- The next tree focuses more on "hard" examples.

## Key parameter: Learning rate

In boosting, models are built one after another, and each new model $f_m(x)$ tries to correct the errors of the previous combined model $F_{m-1}(x)$.

But we don’t add the full correction — we only add a **fraction** of it. That fraction is the **learning rate**:

$$
F_m(x) = F_{m-1}(x) + \eta \cdot f_m(x)
$$

Where:

- $F_m(x)$ = ensemble after adding the m-th tree  
- $f_m(x)$ = prediction of the new tree (the “correction”)  
- $\eta$ = learning rate, with $0 < \eta \le 1$

| Learning Rate    | Number of Trees  | Risk of Overfitting | Behavior                 |
| ---------------- | ---------------- | ------------------- | ------------------------ |
| Small (0.01–0.1) | Large (500–2000) | Low                 | Slow, stable learning    |
| Medium (0.1–0.3) | Moderate         | Medium              | Good default             |
| Large (0.5–1)    | Small            | High                | Jumps too fast, unstable |


XGBoost and LightGBM often recommend:

- η around 0.05–0.3, and

- a lot of trees (hundreds or thousands).


## Popular boosting algorithms:



| Algorithm     | Key Idea | Strengths | Best Use Case |
|---------------|----------|-----------|----------------|
| **AdaBoost** | Increase weights of misclassified points | Simple, intuitive | Clean data, simple models |
| **Gradient Boosting** | Fit residuals (gradients) | Flexible, foundational | General-purpose boosting |
| **XGBoost** | Optimized gradient boosting | Fast, accurate, regularized | Medium/large datasets |
| **LightGBM** | Histogram + leaf-wise growth | Very fast, scalable | Very large datasets |
| **CatBoost** | Categorical feature handling | Minimal preprocessing | Many categorical variables |


## Example implementation


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

gbr_model = GradientBoostingRegressor(
    n_estimators=1000,      # more trees, but shallow
    learning_rate=0.05,    # step size (shrinkage)
    max_depth=3,          # depth of individual trees (weak learners)
    random_state=1
)

gbr_model.fit(X_train, y_train)
gbr_preds = gbr_model.predict(X_valid)
gbr_mae = mean_absolute_error(y_valid, gbr_preds)

print("\nGradient Boosting MAE: {:.3f}".format(gbr_mae))

The `GradientBoostingRegressor` performs even better than the `Random Forest`. [Scikit-Learn's implementation](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html) uses Friedman’s Gradient Boosting algorithm (MART — Multiple Additive Regression Trees), where each stage fits a decision tree to the negative gradient of the loss function.

# Feature Engineering

We just finished training our first machine learning models. To further improve the predictive power of the models we will have to __work on our dataset__.

## Missing Value Imputation


We will start with handling missing values in the data. Most machine learning libraries (including scikit-learn) give an error if we try to build a model using data with missing values. So we'll need to choose a strategy to handle missing values.

We have already used a very simple strategy and dropped all rows containing missing values in the first example. To evaluate different approaches we will first load the full dataset and create a train-test split. (Note: As we cannot apply all imputation functions (e.g., mean) to categorical data we will only use numerical predictions in this simple example.)

In [ ]:
# Load dataset
data = pd.read_csv(melbourne_file_path)

# Target variable
y = data['Price']

# Drop non-numeric variables
melb_predictors = data.drop(['Price'], axis=1)
X = melb_predictors.select_dtypes(exclude=['object'])

# Train-test split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,random_state=0)

### Simple Imputation using the Mean

One popular way to handle missing values is called imputation. Here, we fill in the missing values with some number. For instance, we can fill in the __mean__ value along each column. The imputed value won't be exactly right in most cases, but it usually leads to more accurate models than you would get from dropping the column entirely.

In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
# Imputation
simple_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(simple_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(simple_imputer.transform(X_valid))

# "Repair" column names
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

To evaluate the performance of the approach, we modify our helper function (get_mae) to train and evaluate our model on different datasets:

In [ ]:
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=1)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

In [ ]:
mae_imputation = score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid)
print("MAE using Imputation: {}".format(mae_imputation))

### Advanced Imputation



We see that the imputation approach performs much better compared to the simple solution dropping all rows with NA values.

Imputation is the standard approach, and it usually works well. However, imputed values may be systematically above or below their actual values (which weren't collected in the dataset). Or rows with missing values may be unique in some other way. In that case, your model would make better predictions by considering which values were originally missing.

In the advanced imputation approach, we impute the missing values, as before. And, additionally, for each column with missing entries in the original dataset, we __add a new column__ that shows the __location of the imputed entries__.

In [ ]:
# Make a copy of the original datasets to avoid changing the original data frame
X_train_plus = X_train.copy()
X_valid_plus = X_valid.copy()

# Find all columns with missing values:
cols_with_missing = X_train.columns.values[X_train.isna().sum() > 0]

# Make new columns indicating what will be imputed
for col in cols_with_missing:
    X_train_plus[col + '_was_missing'] = X_train_plus[col].isnull()
    X_valid_plus[col + '_was_missing'] = X_valid_plus[col].isnull()

X_train_plus.head()

In [ ]:
# Imputation
simple_imputer = SimpleImputer()
imputed_X_train_plus = pd.DataFrame(simple_imputer.fit_transform(X_train_plus))
imputed_X_valid_plus = pd.DataFrame(simple_imputer.transform(X_valid_plus))

# "Repair" column names
imputed_X_train_plus.columns = X_train_plus.columns
imputed_X_valid_plus.columns = X_valid_plus.columns

In [ ]:
mae_imputation_advanced = score_dataset(imputed_X_train_plus, imputed_X_valid_plus, y_train, y_valid)
print("MAE using Imputation: {}".format(mae_imputation_advanced))

As we see, advanced imputation does not improve the performance of our model in the problem at hand. In general, advanced imputation will meaningfully improve results in some cases. In other cases, it doesn't help at all.

# Normalization, Standardization, Statistical Transformations

Our dataset may contain attributes with a mixture of scales for various quantities. However, many machine learning methods require the data attributes to have the same scale. 
For example, ``yearBuilt`` is measured in years in our dataset at hand whereas the number of rooms obviously has a much smaller scale.
To avoid having numeric values from different scales we can use two popular data scaling methods: normalization and standardization.
We will implement this step on top of the already imputed dataset from before.

## Normalization

Normalization refers to rescaling numeric attributes into the __range between 0 and 1.__ It is useful to scale the input attributes for a model that relies on the magnitude of values, such as distance measures used in k-nearest neighbors and in the preparation of coefficients in regression.

Using Scikit-learn's ``MinMaxScaler`` we can rescale an attribute according to the following formula:


\begin{equation}
    X = \frac{(X - min(X))}{(max(X) - min(X))}
\end{equation}


In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()


train_X_normalized = pd.DataFrame(scaler.fit_transform(imputed_X_train), 
                                      columns=imputed_X_train.columns, index=imputed_X_train.index)
val_X_normalized = pd.DataFrame(scaler.transform(imputed_X_valid), 
                                    columns=imputed_X_valid.columns, index=imputed_X_valid.index)

train_X_normalized

## Standardization

In contrast to normalization, we could also use standardization for our numerical variables. In this context, standardization refers to shifting the distribution of each attribute to have a mean of zero and a standard deviation of one. It is useful to standardize attributes for a model that relies on the distribution of attributes such as Gaussian processes.
Using Scikit-learn's ```StandardScaler``` we can rescale an attribute according to the following formula:


\begin{equation}
    X = \frac{(X - mean(X))}{\sqrt{var(X)}}
\end{equation}

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

train_X_standardized = pd.DataFrame(scaler.fit_transform(imputed_X_train), 
                                        columns=imputed_X_train.columns, index=imputed_X_train.index)
val_X_standardized = pd.DataFrame(scaler.transform(imputed_X_valid), 
                                      columns=imputed_X_valid.columns, index=imputed_X_valid.index)

train_X_standardized.head()

## Log transforms

Variables can, such as ``Landsize``, span several orders of magnitude. While the vast majority of persons has small sizes of land, a few people have very vast properties. To work with such skewed variables we can use the log transformation. 

__Log transforms__ are useful when applied to skewed distributions as they tend to expand the values which fall in the range of lower magnitudes and tend to compress or reduce the values which fall in the range of higher magnitudes. This tends to make the skewed distribution as normal-like as possible.

In [ ]:
data['Landsize'].describe()

In [ ]:
import numpy as np

train_X_logGains = imputed_X_train.copy()
val_X_logGains = imputed_X_valid.copy()



train_X_logGains['logLandsize'] = np.log1p(train_X_logGains['Landsize'])
val_X_logGains['logLandsize'] = np.log1p(val_X_logGains['Landsize'])

We can see this effect plotting both histograms:

In [ ]:
%matplotlib inline
train_X_logGains[['Landsize', 'logLandsize']].hist();

## Summary

Data rescaling is an important part of data preparation before applying machine learning algorithms. However, it is hard to know whether normalization or standardization of the data will improve the performance of a predictive model in advance. 

A good tip for a practical application is to create rescaled copies of your dataset and evaluate them against each other. This process can quickly show which rescaling method will improve your selected models in the problem at hand. Note that you could (and should) combine these techniques to train powerful models and apply them in real-world problems.

## Exercise 3
We can train our model using the different datasets and feature engineering techniques to evaluate their impact on the model performance. 
Print the score for the different methods we just introduced and compare them.

In [ ]:
# code here

## Encoding Categorical Variables

Until now we only used numerical features for our models. However, valuable information is often encoded in categorical variables (e.g., gender, city, job).

If we simply __plug these categorial variables__ into machine learning models we will get an __error__. Hence, we need to find an appropriate preprocessing to capture the information hidden in categorical variables.

The easiest approach to deal with categorical variables is to __drop__ them from the dataset (that is what we have done before). However, this approach will only produce satisfying results if the dropped columns __did not contain useful information__.

### Label Encoding

<img src="https://raw.githubusercontent.com/GuntherGust/tds2_data/main/images/03/label.png" style="width:60%; float:left;" />



One common approach to handle categorical variables is called label encoding. Here, we assign __each unique value to a different integer (e.g., bad = 0, neutral = 1, good = 2)__.

This assumption makes sense in this example, because there is an indisputable __ranking__ to the categories. Not all categorical variables have a clear ordering in the values, but we refer to those that do as ordinal variables. For tree-based models (like decision trees and random forests), you can expect label encoding to work well with ordinal variables.

For simplicity, we will drop columns with missing values for the following evaluation.

In [ ]:
# Load dataset
data = pd.read_csv(melbourne_file_path)

# Drop NA
data.dropna(axis=0, inplace=True)

# Separate target from predictors
y = data['Price']
X = data.drop(['Price'], axis=1)

# Train-test split
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,random_state=0)

As we do not want to use all categorical variables we focus on those with a __limited number of categories__:

In [ ]:
low_cardinality_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and
                        X_train_full[cname].dtype == "object"]

print(low_cardinality_cols)

...and combine them with the numerical variables:

In [ ]:
# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Keep only selected columns
cols_to_keep = low_cardinality_cols + numerical_cols
X_train = X_train_full[cols_to_keep].copy()
X_valid = X_valid_full[cols_to_keep].copy()

X_train.head()

We can now perform label encoding on our new dataset using the functions provided by Scikit-learn. Subsequently, we can evaluate our approach by using our score_dataset utility function.

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
# Make a copy to protect original data
label_X_train = X_train.copy()
label_X_valid = X_valid.copy()

# Apply label encoder
label_encoder = LabelEncoder()
for col in low_cardinality_cols:
    label_X_train[col] = label_encoder.fit_transform(X_train[col])
    label_X_valid[col] = label_encoder.transform(X_valid[col])

label_X_train.head()

In [ ]:
# Evaluate performance
mae_label_encoding = score_dataset(label_X_train, label_X_valid, y_train, y_valid)
print("MAE using Label Encoding: {}".format(mae_label_encoding))

This result is considerably better than the model that relied only on numerical variables without imputation (MAE 191.669 USD).

### One-hot Encoding

<img src="https://raw.githubusercontent.com/GuntherGust/tds2_data/main/images/03/onehot.png" style="width:70%; float:left;" />



One-hot encoding creates new binary columns indicating the presence (or absence) of each possible value in the original data.

In contrast to label encoding, one-hot encoding does __not assume an ordering__ of the categories. Thus, you can expect this approach to work particularly well if there is no clear ordering in the categorical data. We refer to categorical variables without an intrinsic ranking as __nominal variables__.

One-hot encoding generally does not perform well if the categorical variable takes on a large number of values (i.e., you generally __won't use__ it for variables taking on __many__ (e.g. more than 15) __different values__).

Again, we can use Scikit-learn functions to implement one-hot encodings:

In [ ]:
from sklearn.preprocessing import OneHotEncoder

Let's first look at the original values of the categorial variables before their transformation:

In [ ]:
X_train[low_cardinality_cols].describe()

In [ ]:
# Apply one-hot encoder to each column with categorical data
one_hot_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
one_hot_cols_train = pd.DataFrame(one_hot_encoder.fit_transform(X_train[low_cardinality_cols]))
one_hot_cols_valid = pd.DataFrame(one_hot_encoder.transform(X_valid[low_cardinality_cols]))

# Repair index
one_hot_cols_train.index = X_train.index
one_hot_cols_valid.index = X_valid.index

Let's look at the result of the one-hot encoding:

In [ ]:
one_hot_cols_train.head()

Let's train the model:

In [ ]:
# Remove categorical columns and replace with one-hot encoding
num_X_train = X_train.drop(low_cardinality_cols, axis=1)
num_X_valid = X_valid.drop(low_cardinality_cols, axis=1)
one_hot_X_train = pd.concat([num_X_train, one_hot_cols_train], axis=1)
one_hot_X_valid = pd.concat([num_X_valid, one_hot_cols_valid], axis=1)

print(one_hot_X_train.head())
# Evaluate performance
one_hot_encoding = score_dataset(one_hot_X_train.to_numpy(), one_hot_X_valid.to_numpy(), y_train, y_valid)
print("MAE using One-hot Encoding: {}".format(one_hot_encoding))

This is a small improvement in comparison to the label encoded data (MAE 181.607 USD).

## Exercise 1
Use standardized numeric features like in the previous section and combine it with OneHot Encoding and Label Encoding for training the model.

The final results should have two cases: 

- One-hot encoded categorical + standardized numeric
- Label encoded categorical + standardized numeric

For each of the considered categorical columns, does it make more sense to use label encoding or one-hot encoding from a data perspective?

In [ ]:
# code here

## Creating Model Pipelines

Up to now, we learned how to prepare our datasets, train, tune, and evaluate powerful models. However, we wrote lots of code and functions to perform all the required tasks. Scikit-learn pipelines are a simple way to keep our data preprocessing and modeling code organized. Specifically, a pipeline bundles preprocessing and modeling steps so we can use the whole bundle as if it were a single step.

Using pipelines provides multiple benefits:
* Cleaner Code
* Fewer Bugs
* Easier to Productionize
* More Options for Model Validation

We will build a pipeline using all numerical variables as well as the low cardinatlity categorical variables

In [ ]:
# Load dataset
data = pd.read_csv(melbourne_file_path)

# Separate target from predictors
y = data['Price']
X = data[cols_to_keep]

# Train-test split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,random_state=0)
X_train.head()

Writing a pipeline in Scikit-learn can be broken down into 3 steps:
1. Define preprocessing steps
2. Define the model
3. Create and evaluate the pipeline

### Define Preprocessing steps

We use the ``ColumnTransformer`` class to bundle together different preprocessing steps. To this end, we will impute missing values in the numerical columns and impute missing values and use one-hot encoding in the categorical columns.

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [ ]:
# Preprocessing numerical columns
numerical_transformer = SimpleImputer(strategy='mean')

# Preprocessing categorical columns
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle both preprocessors
preprocessor = ColumnTransformer(transformers=[
    ('num', numerical_transformer, numerical_cols),
    ('cat', categorical_transformer, low_cardinality_cols)
])

### Define the Model
Next, we define a random forest model.

In [ ]:
model = RandomForestRegressor(n_estimators=100, random_state=1)

### Create and Evaluate the Pipeline

Finally, we use the ``Pipeline`` class to define a pipeline that bundles the preprocessing and modeling steps.

In [ ]:
# Bundle preprocessing and modeling code in a pipeline
complete_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model)
])

# Preprocess the raw training data and fit the model
complete_pipeline.fit(X_train, y_train)

# Preprocess the raw validation data and make predictions
preds = complete_pipeline.predict(X_valid)

# Evaluate the model
score = mean_absolute_error(y_valid, preds)
print("MAE using the complete pipeline: {}".format(score))

 There are a few important things to notice:

* With the pipeline, we preprocess the training data and fit the model in a __single line of code__. (In contrast, without a pipeline, we have to do imputation, one-hot encoding, and model training in separate steps. This becomes especially messy if we have to deal with both numerical and categorical variables!)
* With the pipeline, we supply the __unprocessed features in X_valid to the predict()__ command, and the pipeline __automatically preprocesses__ the features before generating predictions. (However, without a pipeline, we have to remember to preprocess the validation data before making predictions.)

# Exercise 5

Extend the pipeline by including the normalization step for the numerical features.

In [ ]:
# code here

# Unsupervised Learning

## K-means Clustering


K-Means clustering is the __most popular unsupervised__ learning algorithm. It is used when we have unlabelled data which is data without defined categories or groups. The algorithm follows an easy or simple way to classify a given data set through a certain number of __clusters, fixed apriori.__ K-Means algorithm works iteratively to assign each data point to one of K groups based on the features that are provided. Data points are clustered based on __feature similarity.__



<img src="https://raw.githubusercontent.com/vhaus63/ids_data/main/kmeans.png" style="width:80%" />

### Goal

Given a __dataset__ and a __number of clusters *k*__, K-Means tries to:

- Partition the data into *k* groups
- Minimize the **within-cluster sum of squared distances** (WCSS)

In other words:
> Find cluster centers so that points within a cluster are as close as possible to each other.

### The K-Means Algorithm

1. **Initialization**  
   Choose *k* initial cluster centers (centroids), either randomly or using k-means++.

2. **Assignment Step**  
   For each data point:
   - Assign it to the nearest centroid (measured by Euclidean distance).

3. **Update Step**  
   For each cluster:
   - Recompute the centroid as the **mean** of all points assigned to it.

4. **Repeat**  
   - Alternate between *assignment* and *update* steps  
   - Stop when centroids no longer change significantly (convergence)

### K-means Algorithm

<img src="https://raw.githubusercontent.com/vhaus63/ids_data/main/kmeans_algorithm.png" style="width:50%" />

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/vhaus63/ids_data/main/Mall_Customers.csv')
df.head()

In [ ]:
X = df[['Annual Income (k$)', 'Spending Score (1-100)']]

In [ ]:
from lets_plot import *

LetsPlot.setup_html()

plt = (
    ggplot(X, aes(x='Annual Income (k$)', y='Spending Score (1-100)'))
    + geom_point()
    + ggsize(700, 400)
    + labs(
        x='Annual Income (k$)',
        y='Spending Score (1-100)',
        title='Scatter Plot of Annual Income vs. Spending Score'
    )
    + theme_minimal()
)
plt

Now we can fit the model. We choose 5 clusters for the beginning, based on our visual inspection of the dataset and its distribution.

In [ ]:
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="sklearn.cluster._kmeans")

kmeans = KMeans(n_clusters=5, random_state=0, n_init="auto")
kmeans.fit(X)
kmeans.cluster_centers_

Let's plot the cluster membership for the dataset:

In [ ]:
clusters_df = pd.concat([X, pd.Series(kmeans.labels_, name='Cluster')], axis=1)

# Scatter plot for clusters
p = ggplot(clusters_df) + \
    geom_point(aes(x='Annual Income (k$)', y='Spending Score (1-100)', color='Cluster')) + \
    scale_color_discrete(name='Cluster') + \
    labs(x='Annual Income (k$)', y='Spending Score (1-100)', color='Cluster')

p


### Parameters to tune the model

#### 1. k - the amount of clusters:

We can estimate a suitable number of clusters __using inertia__ and the __elbow method:__

__WCSS (Within-Cluster Sum of Squares),__ also known as __inertia,__ measures the sum of squared distances between each data point and the centroid of its assigned cluster. The centroid is the mean position of all the points in the cluster.
It provides a measure of the "costs" of the k means model and can help to identify the right choice of k.

In [ ]:
kmeans.inertia_

In [ ]:
def elbow_method(X, max_k = 10):
    costs = []
    for k in range(2, max_k):
        model = KMeans(n_clusters=k, random_state=0, n_init="auto")
        model.fit(X)
        costs.append(model.inertia_)

    costs_df = pd.DataFrame({
        'K': range(2, max_k),  # Cluster counts (k)
        'Cost': costs  # The corresponding cost (inertia)
    })
    
    # Plotting the elbow curve
    p = (ggplot(costs_df, aes(x='K', y='Cost'))
        + geom_line()
        + geom_point()
        + labs(x='Number of Clusters (k)', y='Cost (Inertia)'))
    return p

## Mentimeter (Elbow method)

How many clusters to choose based on the following plot? Answer on [Mentimeter](https://www.menti.com/alurte2x2dno). 

In [ ]:
elbow_method(X)

The elbow method should be utilized with care since it is both __subjective and unreliable.__ In many practical applications, the choice of an "elbow" is highly ambiguous as the plot does not contain a sharp elbow. Because the two axes (the number of clusters and the remaining variance) have no semantic relationship, the plot is sensitive to the __parameter range.__

#### 2. The distance measure

The choice of a distance metric can affect the clustering results, depending on the nature of the data.

Some distance measures are the following:

- **Euclidean Distance**: $d(p,q)=\sqrt(\sum_{i=1}^n(p_i-q_i)^2)$.

It is the most widely used distance metric that reflects our __intuitive__ way of understanding distances and works well with __spherical clusters__ in continuous spaces. However, it is __sensitive to scale,__ so features with a wide range can dominate the clustering results, which might not always be desired. It also assumes some circle-like underlying structure of the clusters.

- **Manhattan distance**: $d(p,q)=\sum_{i=1}^n\vert p_i-q_i\vert$.

The Manhattan distance (or L1 Norm) handles different scales in features better than the euclidean distance (in contrast to which no square is introduced) and is therefore __less affected from outliers.__ It works very well on grid-like data. This also means, that it assumes an alignment of the clusters with the axes: it is __not invariant under rotation.__

- **Hamming Distance**: Number of positions where $p_i\neq q_i$.

The Hamming distance is an effective and simple metric for categorical data. However, due to its definition, it is not applicable to continuous data.

- **Chi-Squared Distance**: $d(p,q)=\sum_{i=1}^n\frac{(p_i-q_i)^2}{p_i+q_i}$, where $p_i+q_i>0$.

It is commonly used in clustering tasks, particularly for histograms and frequency data. As visible from the formula, it accounts for the relative difference between features. It is well-suited for __comparing distributions or normalized data.__ The Chi-Squared distance can only be applied to non-negative data and it is very sensitive to small values. Additionally, like the Manhattan Distance, it is not rotation invariant.

### Excursus: Effect of different distance metrics (not relevant for exam)

Sklearns' `kMeans` does not natively support the use of other distance metrices than the euclidean distance. To demonstrate the effect of changing this measure, we will use the `PyClustering` module.

In [ ]:
from pyclustering.cluster.kmeans import kmeans
from pyclustering.cluster.center_initializer import random_center_initializer
from pyclustering.utils import distance_metric, type_metric

In [ ]:
X_list = [[1, 2], [1, 4], [1, 0], [4, 2], [4, 4], [4, 0], [10, 2], [10, 4], [10, 0]]

metrics = [
    ('Squared Euclidean', distance_metric(type_metric.EUCLIDEAN_SQUARE)),
    ('Chi Square', distance_metric(type_metric.CHI_SQUARE)),
    ('Manhattan', distance_metric(type_metric.MANHATTAN))
]

for metric_name, metric in metrics:
    print(f"Clustering with {metric_name} Distance:")

    initial_centers = random_center_initializer(X_list, 5, random_state=0).initialize()

    kmeans_instance = kmeans(X_list, initial_centers=initial_centers, metric=metric)
    kmeans_instance.process()
    
    centers = kmeans_instance.get_centers()
    clusters = kmeans_instance.get_clusters()
    
    print(f"  Cluster Centers: {centers}")
    print(f"  Clusters: {clusters}\n")

    cluster_data = []
    for cluster_idx, cluster in enumerate(clusters):
        for point_idx in cluster:
            x, y = X_list[point_idx]
            cluster_data.append({'x': x, 'y': y, 'label': cluster_idx})
    center_data = [{'x': center[0], 'y': center[1], 'label': f'C{idx}'} for idx, center in enumerate(centers)]
    
    # Convert to dataframes for plotting
    cluster_df = pd.DataFrame(cluster_data)
    center_df = pd.DataFrame(center_data)
    
    # Plot clusters and centers
    plot = (ggplot(cluster_df, aes('x', 'y', shape='label',  color='label'))
            + scale_x_continuous(breaks = np.arange(0, 11,  1))
            + scale_y_continuous(breaks = np.arange(0, 10,  1))
            + geom_point(size=8)
            + scale_shape(name='Cluster')
            + coord_cartesian(ylim=(-1, 7), xlim = (0,11)) 
            + scale_color_discrete(name='Cluster')
            + ggtitle(f'Clusters using {metric_name} Distance')            
            + theme_minimal()
           )
    display(plot)

__Choosing the right distance metric__ is an important step in using clustering algorithms like KMeans. The choice depends on the type of data you're working with-__continuous, categorical, or binary__, its inherent structure and relationships within your data and the specific requirements of your problem.

### More on k-means clustering

- For an alternative explanation of k-means, please watch [this video](https://www.youtube.com/watch?v=4b5d3muPQmA). <span style="color:red">__This content is relevant for the exam!__</span>
- For a formal coverage see [Hastie et al: Elements of Statistical Learning, Chapter 13](https://www.sas.upenn.edu/~fdiebold/NoHesitations/BookAdvanced.pdf)

## Further Clustering Algorithms

In addition to the k-Means clustering algorithm, there are several other options to choose from. Here is an overview on how they deal with different datsets:

<img src="https://raw.githubusercontent.com/vhaus63/ids_data/main/different_clustering_algos.png" style="width:70%" />

You can see very well here that kMeans performs very well on __spherical clusters,__ but __not at all on non-convex clusters.__ For these shapes, other algorithms like DBSCAN are recommendet. If you are interested in other clustering algorithms, please refer to the [documentation](https://scikit-learn.org/1.5/auto_examples/cluster/plot_cluster_comparison.html).

## Summary

Let's summarize the clustering using [Mentimeter](https://www.menti.com/alurte2x2dno).


## Next lecture: Descriptive Models!

<img src="https://raw.githubusercontent.com/vhaus63/ids_data/main/ao_modeling.png" style="width:80%" />
